<a href="https://colab.research.google.com/github/AprilVilmin/Neural_Network_Charity_Analysis/blob/main/AlphabetSoupCharity_Optimzation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
# https://stackoverflow.com/questions/40389018/dropping-multiple-columns-from-a-data-frame-using-python
application_df.drop(['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS', 'AFFILIATION'], axis=1, inplace=True)

In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df.APPLICATION_TYPE.value_counts()
application_counts

In [ ]:
# Visualize the value counts of APPLICATION_TYPE
application_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df.CLASSIFICATION.value_counts()
class_counts

In [ ]:
# Visualize the value counts of CLASSIFICATION
class_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ..?
replace_class= list(class_counts[class_counts < 500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

In [ ]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

In [ ]:
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 300
hidden_nodes_layer2 = 250
hidden_nodes_layer3 = 200
hidden_nodes_layer4 = 150
hidden_nodes_layer5 = 100

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
import keras

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint('model{epoch:08d}.h5', period=5) 
nn.fit(X_train_scaled, y_train, callbacks=[checkpoint])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 28s 35ms/step - loss: 0.6096 - accuracy: 0.6541
Epoch 2/50
804/804 [==============================] - 26s 32ms/step - loss: 0.6095 - accuracy: 0.6530
Epoch 3/50
804/804 [==============================] - 22s 28ms/step - loss: 0.6076 - accuracy: 0.6563
Epoch 4/50
804/804 [==============================] - 22s 27ms/step - loss: 0.6081 - accuracy: 0.6562
Epoch 5/50
804/804 [==============================] - 21s 26ms/step - loss: 0.6077 - accuracy: 0.6546
Epoch 6/50
804/804 [==============================] - 23s 28ms/step - loss: 0.6075 - accuracy: 0.6569
Epoch 7/50
 31/804 [>.............................] - ETA: 21s - loss: 0.5845 - accuracy: 0.6815

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# 10: Save and export your results to an HDF5 file, and name it AlphabetSoupCharity.h5
# Copied code from https://github.com/PopeScooby/Neural_Network_Charity_Analysis/blob/main/AlphabetSoupCharity.ipynb
nn.save("AlphabetSoupCharity_Optimzation2.h5")